In [1]:
import os
import json
import pandas as pd
import pdfplumber
import re
from datetime import datetime

class QuestionnaireSectionExtractor:
    """Class to extract sections and tables from questionnaire PDFs"""
    
    def __init__(self, output_dir=None):
        """Initialize the extractor with an optional output directory"""
        self.output_dir = output_dir
    
    def extract_from_pdf(self, pdf_file):
        """Extract sections and tables from the PDF"""
        print(f"Extracting sections and tables from: {pdf_file}")
        
        # Store extracted data
        sections = []
        current_section = None
        current_section_id = None
        page_heights = {}
        
        try:
            with pdfplumber.open(pdf_file) as pdf:
                # First pass: identify sections and their boundaries
                for page_num, page in enumerate(pdf.pages):
                    page_number = page_num + 1
                    page_heights[page_number] = page.height
                    print(f"Processing page {page_number} of {len(pdf.pages)}")
                    
                    # Extract text
                    text = page.extract_text() or ""
                    
                    # Find section headers using regex
                    # Pattern for section identifiers like (1.1) or 1.1 
                    section_pattern = re.compile(r'(?:\()?(\d+\.\d+(?:\.\d+)?)(?:\))?\s+([^\n\.]+)')
                    
                    for match in section_pattern.finditer(text):
                        section_id = match.group(1)
                        section_title = match.group(2).strip()
                        
                        # Skip very short or likely non-section titles
                        if len(section_title) < 3 or section_title.isdigit():
                            continue
                        
                        # If we have a current section, finalize it before starting a new one
                        if current_section is not None:
                            sections.append({
                                "section_id": current_section_id,
                                "section_title": current_section,
                                "start_page": current_section_page,
                                "end_page": page_number - 1 if page_number > current_section_page else page_number,
                                "start_position": current_section_position,
                                "end_position": match.start(),
                                "tables": []  # Will be filled in second pass
                            })
                        
                        # Start new section
                        current_section = section_title
                        current_section_id = section_id
                        current_section_page = page_number
                        current_section_position = match.start()
                
                # Add the last section if there is one
                if current_section is not None:
                    sections.append({
                        "section_id": current_section_id,
                        "section_title": current_section,
                        "start_page": current_section_page,
                        "end_page": len(pdf.pages),
                        "start_position": current_section_position,
                        "end_position": float('inf'),
                        "tables": []
                    })
                
                # Second pass: extract tables and assign to sections
                for page_num, page in enumerate(pdf.pages):
                    page_number = page_num + 1
                    
                    # Extract tables with standard settings
                    tables = page.extract_tables()
                    
                    # Process tables on this page
                    for table_idx, table in enumerate(tables):
                        if not table or len(table) == 0:
                            continue
                        
                        # Get table bounds
                        table_bounds = self._get_table_bounds(page, table)
                        
                        # Find which section this table belongs to
                        section_idx = self._find_section_for_table(sections, page_number, table_bounds)
                        
                        if section_idx is not None:
                            # Clean and process table
                            processed_table = self._process_table(table)
                            
                            # Add to section's tables
                            sections[section_idx]["tables"].append({
                                "page": page_number,
                                "table_index": table_idx,
                                "bounds": table_bounds,
                                "data": processed_table
                            })
                    
                    # Try alternative table extraction for complex tables if needed
                    if not tables:
                        alt_tables = page.extract_tables({
                            "vertical_strategy": "text", 
                            "horizontal_strategy": "text",
                            "intersection_tolerance": 5
                        })
                        
                        for table_idx, table in enumerate(alt_tables):
                            if not table or len(table) == 0:
                                continue
                            
                            # Get table bounds (approximate for alternative tables)
                            table_bounds = self._get_table_bounds(page, table, is_alt=True)
                            
                            # Find which section this table belongs to
                            section_idx = self._find_section_for_table(sections, page_number, table_bounds)
                            
                            if section_idx is not None:
                                # Clean and process table
                                processed_table = self._process_table(table)
                                
                                # Add to section's tables
                                sections[section_idx]["tables"].append({
                                    "page": page_number,
                                    "table_index": f"alt_{table_idx}",
                                    "bounds": table_bounds,
                                    "data": processed_table
                                })
            
            # Post-process sections to identify table types and relationships
            enhanced_sections = self._enhance_sections(sections)
            
            # Create structured output
            result = {
                "document": {
                    "filename": os.path.basename(pdf_file),
                    "extraction_date": datetime.now().isoformat(),
                    "sections": enhanced_sections
                }
            }
            
            return result
            
        except Exception as e:
            print(f"Error extracting sections and tables: {str(e)}")
            import traceback
            traceback.print_exc()
            return {"error": str(e)}
    
    def _get_table_bounds(self, page, table, is_alt=False):
        """Get the bounds of a table on the page"""
        if is_alt:
            # For alternative extraction, we don't have precise bounds
            # Make an estimate based on the page dimensions
            return {
                "x0": 0,
                "top": 0,
                "x1": page.width,
                "bottom": page.height
            }
        
        # Try to find the table on the page
        tables = page.find_tables()
        
        for t in tables:
            # Check if this is our table by comparing content
            # Convert both to string representations for comparison
            table_str = str([[str(cell) for cell in row] for row in table])
            found_table_str = str([[str(cell) for cell in row] for row in t.extract()])
            
            # If content matches approximately
            if len(table_str) > 0 and len(found_table_str) > 0 and (
                table_str[:100] in found_table_str or found_table_str[:100] in table_str):
                
                return {
                    "x0": t.bbox[0],
                    "top": t.bbox[1],
                    "x1": t.bbox[2],
                    "bottom": t.bbox[3]
                }
        
        # If not found, make an estimate
        return {
            "x0": 0,
            "top": 0,
            "x1": page.width,
            "bottom": page.height
        }
    
    def _find_section_for_table(self, sections, page, table_bounds):
        """Find which section a table belongs to based on page and position"""
        for idx, section in enumerate(sections):
            # If table is on a page between section start and end
            if section["start_page"] <= page <= section["end_page"]:
                # If it's on the start page, check if it's after the section start
                if page == section["start_page"] and table_bounds["top"] < section["start_position"]:
                    continue
                
                # If it's on the end page (and not the last section), check if it's before the section end
                if (page == section["end_page"] and idx < len(sections) - 1 and 
                    table_bounds["bottom"] > section["end_position"]):
                    continue
                
                return idx
        
        # If no matching section found and we have sections, assign to the last section
        if sections:
            return len(sections) - 1
        
        return None
    
    def _process_table(self, table):
        """Clean and process a table"""
        # Convert to pandas DataFrame
        df = pd.DataFrame(table)
        
        # Handle empty DataFrame
        if df.empty:
            return []
        
        # Drop completely empty rows and columns
        df = df.dropna(how='all').dropna(axis=1, how='all')
        
        # Handle column names
        if df.columns.duplicated().any():
            # Make column names unique
            df.columns = [f"{col}_{i}" if i > 0 else col 
                         for i, col in enumerate(pd.Series(df.columns).groupby(df.columns).cumcount())]
        
        # Clean cell values
        for col in df.columns:
            df[col] = df[col].map(lambda x: str(x).strip() if x is not None else "", na_action='ignore')
        
        # Fill NaN values
        df = df.fillna("")
        
        # Convert to list of dictionaries
        return df.to_dict('records')
    
    def _enhance_sections(self, sections):
        """Enhance sections with table classification and relationships"""
        enhanced_sections = []
        
        for section in sections:
            # Skip sections with no tables
            if not section["tables"]:
                enhanced_sections.append(section)
                continue
            
            # Get all tables in the section
            tables = section["tables"]
            
            # Classify tables
            for table in tables:
                # Convert table data back to DataFrame for analysis
                df = pd.DataFrame(table["data"])
                
                # Skip empty tables
                if df.empty:
                    table["type"] = "empty"
                    continue
                
                # Analyze table content
                table["type"] = self._classify_table(df)
                
                # For question-answer tables, try to extract structure
                if table["type"] in ["question_answer", "form_field"]:
                    structure = self._extract_table_structure(df, table["type"])
                    table["structure"] = structure
            
            # Find relationships between tables
            related_tables = self._find_table_relationships(tables)
            
            # Update the section
            enhanced_section = section.copy()
            enhanced_section["tables"] = tables
            enhanced_section["table_relationships"] = related_tables
            
            enhanced_sections.append(enhanced_section)
        
        return enhanced_sections
    
    def _classify_table(self, df):
        """Classify table based on content and structure"""
        # Convert all values to strings for analysis
        str_values = [[str(cell).strip() for cell in row] for row in df.values]
        flat_values = [cell for row in str_values for cell in row if cell]
        
        # Check for common patterns
        
        # Check if this looks like a question details table
        question_details_terms = ["Question details", "Change from last year", "Rationale", "Response options"]
        if any(term in ' '.join(flat_values) for term in question_details_terms):
            return "question_details"
        
        # Check if this looks like a response table
        response_terms = ["Response", "Select from", "Text field", "Numeric field"]
        if any(term in ' '.join(flat_values) for term in response_terms):
            return "response_table"
        
        # Check if this is a horizontal vs vertical table
        # Horizontal tables typically have numbered columns (1, 2, 3...)
        first_row = str_values[0] if str_values else []
        has_numeric_headers = any(cell.isdigit() for cell in first_row)
        
        if has_numeric_headers:
            return "horizontal_table"
        
        # Check if this looks like a form field (key-value pairs)
        if df.shape[1] == 2:
            # Check if first column has consistent keys
            first_col = [str(val).strip() for val in df.iloc[:, 0]]
            if all(val and not val.isdigit() for val in first_col if val):
                return "form_field"
        
        # Check if this is a guidance table
        guidance_terms = ["General", "guidance", "Requested content", "Note:", "Example:"]
        if any(term in ' '.join(flat_values) for term in guidance_terms):
            return "guidance"
        
        # Default to general table
        return "general_table"
    
    def _extract_table_structure(self, df, table_type):
        """Extract structured data from table based on its type"""
        structure = {}
        
        if table_type == "form_field" and df.shape[1] >= 2:
            # Extract key-value pairs
            fields = []
            for _, row in df.iterrows():
                key = str(row.iloc[0]).strip()
                value = str(row.iloc[1]).strip() if df.shape[1] > 1 else ""
                
                if key:
                    # Check if value contains options
                    options = []
                    if "Select from:" in value:
                        options_text = value.split("Select from:")[1].strip()
                        # Extract options (typically bullet points)
                        options = [opt.strip(' •\n') for opt in re.split(r'[•□]\s*|\n+', options_text) 
                                   if opt.strip()]
                    
                    fields.append({
                        "field_name": key,
                        "field_value": value,
                        "options": options
                    })
            
            structure["fields"] = fields
            
        elif table_type == "question_answer" or table_type == "horizontal_table":
            # Analyze horizontal structure
            # Check if first row might be headers
            if df.shape[0] > 0:
                headers = df.iloc[0].tolist()
                headers = [str(h).strip() for h in headers]
                
                # Skip if headers are empty or all numeric
                if not all(h.isdigit() or not h for h in headers):
                    structure["headers"] = headers
                    
                    # If there are meaningful headers, extract columns
                    columns = []
                    for col_idx, header in enumerate(headers):
                        if col_idx >= df.shape[1]:
                            continue
                            
                        # Get column values (skip header row)
                        values = df.iloc[1:, col_idx].tolist()
                        values = [str(v).strip() for v in values]
                        
                        columns.append({
                            "header": header,
                            "values": values
                        })
                    
                    structure["columns"] = columns
        
        return structure
    
    def _find_table_relationships(self, tables):
        """Find relationships between tables in a section"""
        relationships = []
        
        # Group tables by page
        tables_by_page = {}
        for idx, table in enumerate(tables):
            page = table["page"]
            if page not in tables_by_page:
                tables_by_page[page] = []
            tables_by_page[page].append((idx, table))
        
        # For each page, find tables that might be related
        for page, page_tables in tables_by_page.items():
            # Sort by vertical position
            page_tables.sort(key=lambda t: t[1]["bounds"]["top"])
            
            # Check for adjacent tables that might be related
            for i in range(len(page_tables) - 1):
                idx1, table1 = page_tables[i]
                idx2, table2 = page_tables[i + 1]
                
                # Check if tables are adjacent
                if (abs(table2["bounds"]["top"] - table1["bounds"]["bottom"]) < 50 or
                    table1["type"] == "question_details" and table2["type"] == "response_table"):
                    
                    relationships.append({
                        "table1_index": idx1,
                        "table2_index": idx2,
                        "relationship_type": "adjacent"
                    })
                
                # Check for question-answer relationship
                if (table1["type"] == "question_details" and 
                    table2["type"] in ["response_table", "form_field", "horizontal_table"]):
                    
                    relationships.append({
                        "table1_index": idx1,
                        "table2_index": idx2,
                        "relationship_type": "question_answer"
                    })
        
        return relationships
    
    def convert_to_json(self, pdf_file, save_output=True):
        """Process a PDF file and convert to structured JSON"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base_filename = os.path.splitext(os.path.basename(pdf_file))[0]
        
        # Create output directory if needed
        if self.output_dir and not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)
        
        # Extract sections and tables
        result = self.extract_from_pdf(pdf_file)
        
        # Save JSON file
        if save_output and self.output_dir:
            json_file = os.path.join(self.output_dir, f"{base_filename}_sections_{timestamp}.json")
            with open(json_file, 'w', encoding='utf-8') as f:
                json.dump(result, f, indent=2, ensure_ascii=False)
            print(f"\nJSON output saved to: {json_file}")
        
        # Print summary
        num_sections = len(result.get("document", {}).get("sections", []))
        total_tables = sum(len(section.get("tables", [])) for section in result.get("document", {}).get("sections", []))
        print(f"\nExtraction complete!")
        print(f"Found {num_sections} sections with {total_tables} tables")
        
        return result

# Example usage
if __name__ == "__main__":
    # Path to your PDF
    pdf_file = r'pdf_files\CDP_2024_Corporate_Questionnaire_Guidance_Modules_1-6.pdf'
    
    # Create output directory
    output_dir = "pdf_extractions"
    os.makedirs(output_dir, exist_ok=True)
    
    # Initialize extractor
    extractor = QuestionnaireSectionExtractor(output_dir=output_dir)
    
    # Convert PDF to JSON
    result = extractor.convert_to_json(pdf_file)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Extracting sections and tables from: pdf_files\CDP_2024_Corporate_Questionnaire_Guidance_Modules_1-6.pdf


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Processing page 1 of 368
Processing page 2 of 368
Processing page 3 of 368
Processing page 4 of 368
Processing page 5 of 368
Processing page 6 of 368
Processing page 7 of 368
Processing page 8 of 368
Processing page 9 of 368
Processing page 10 of 368
Processing page 11 of 368
Processing page 12 of 368
Processing page 13 of 368
Processing page 14 of 368
Processing page 15 of 368
Processing page 16 of 368
Processing page 17 of 368
Processing page 18 of 368
Processing page 19 of 368
Processing page 20 of 368
Processing page 21 of 368
Processing page 22 of 368
Processing page 23 of 368
Processing page 24 of 368
Processing page 25 of 368
Processing page 26 of 368
Processing page 27 of 368
Processing page 28 of 368
Processing page 29 of 368
Processing page 30 of 368
Processing page 31 of 368
Processing page 32 of 368
Processing page 33 of 368
Processing page 34 of 368
Processing page 35 of 368
Processing page 36 of 368
Processing page 37 of 368
Processing page 38 of 368
Processing page 39 of

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def


JSON output saved to: pdf_extractions\CDP_2024_Corporate_Questionnaire_Guidance_Modules_1-6_sections_20250424_131828.json

Extraction complete!
Found 451 sections with 2093 tables
